# Controller analysis

This Jupyter Notebook analyses the behaviour of resulting reinforcement learning (RL) controllers. Three types of controllers are analysed. Two RL controllers trained with different reward functions: an additive and multiplicative approach; and a rule-based controller.

## Variables

The following variables are present in this Jupyter Notebook:

- `additive_path`: A string variable with the value `'../data/benchmark/test/additive-0.99'`.
- `additive_pen_df`: A pandas DataFrame variable with 172,800 rows and 43 columns. It contains data related to air temperature, CO2 concentration, humidity, fruit weight, fruit harvest, PAR, and other variables.
- `axes`: A numpy array of Axes objects used for plotting.
- `bar_width`: A float variable with the value 0.4.
- `colors`: A list of RGB tuples representing colors used for plotting.
- `control_variables`: A list of strings representing control variables.
- `cost`: A string variable with the value `'Cumulative gas'`.
- `cumulative_harvest_rb`: A pandas DataFrame variable with 60 rows and 2 columns. It contains episode numbers and corresponding fruit harvest values for a rule-based model.
- `cumulative_harvest_rl`: A pandas DataFrame variable with 60 rows and 2 columns. It contains episode numbers and corresponding fruit harvest values for a reinforcement learning model.
- `cumulatives`: A list of strings representing cumulative variables.
- `dfs`: A list of pandas DataFrame variables containing time-series data.
- `episode`: An integer variable with the value 0.
- `fig`: A matplotlib Figure object used for plotting.
- `i`: An integer variable with the value 3.
- `labels`: A list of strings representing labels for different models.

## Workflow

The notebook includes various code cells that perform tasks such as data loading, data manipulation, plotting, and analysis. The code cells are organized in a sequential manner, and each cell can be executed independently.

The notebook starts with importing necessary modules and defining functions. It then proceeds with loading data, adjusting time, and performing calculations on the data. Finally, it includes code for generating plots and visualizing the results.

The markdown cells in the notebook are used for providing explanations, documenting the code, and summarizing the results.

Overall, this notebook provides a comprehensive workflow for analyzing and visualizing time-series data using pandas, matplotlib, and other libraries.

In [ ]:
# required imports
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import cmcrameri.cm as cm
from scipy.stats import normaltest, mannwhitneyu, ttest_ind

In [ ]:
### Latex font in plots
plt.rcParams['font.serif'] = "cmr10"
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24

plt.rcParams['legend.fontsize'] = 24
plt.rcParams['legend.loc'] = 'upper right'
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams["axes.grid"] = False
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['axes.linewidth'] = 4   # Default for all spines
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
# plt.rcParams['text.usetex'] = True
plt.rcParams['xtick.major.size'] = 6  # Thicker major x-ticks
plt.rcParams['xtick.major.width'] = 2  # Thicker major x-
plt.rcParams['ytick.major.size'] = 6  
plt.rcParams['ytick.major.width'] = 2 
plt.rc('axes', unicode_minus=False)


# Helper function for making all the relevant plots

In [ ]:
def load_data(data_path, data_file):
    """
    Load the data from a CSV file, given data_path name and data_file.

    Args:
        data_path (str): the path to the data file
        data_file (str): the name of the data file
    
    Returns: (pd.DataFrame): the data
    """
    df = pd.read_csv(os.path.join(data_path, data_file))
    return df

def compute_profit_eps(df):
    """
    Compute the mean profit per episode.

    Args:
        df (pd.DataFrame): the DataFrame containing the data

    Returns:
        pd.Dataframe: averaged profits per episode
    """
    # compute profit per episode
    N = (df[df['episode'] == 0]).shape[0]
    profits_per_episode = df[['Profits', 'episode']].groupby('episode').sum().reset_index()
    return profits_per_episode


def aggregate_data(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Function that computes statistics for violations and profits per episode.
    This function takes in a DataFrame and the name of the column to be used for violations.

    Args:
    - df: the DataFrame
    - column: the name of the column to be used for violations

    Returns:
    - episode: the episode number
    - Profits: the total profits for the episode
    - CO2 Violation Time (%): the percentage of time with CO2 violations
    - CO2 Violation (ppm): the average magnitude of CO2 violations
    """
    N = (df[df['episode'] == 0]).shape[0]
    profits_per_episode = df[['Profits', 'episode']].groupby('episode').sum().reset_index()
    # CO2 violation time per episode, considering each row as 5 minutes
    co2_violation_time_updated = df[df[column] > 0].groupby('episode').size()/N*100 # % of time with violation

    avg_co2_violation_magnitude_updated = df[df[column] > 0].groupby('episode')[column].mean()
    # Combine the updated results into a summary DataFrame
    summary_df_updated = pd.DataFrame({
        f'Time within boundary (%)': co2_violation_time_updated,
        f'{column} (abs)': avg_co2_violation_magnitude_updated,
    }).reset_index()

    # Create a DataFrame of all unique episodes to ensure all are represented
    all_episodes_df = pd.DataFrame(df['episode'].unique(), columns=['episode'])

    # Merge the summary of violations with the complete list of episodes
    # This ensures episodes with no violations are included, filling missing values appropriately
    full_summary_df = pd.merge(all_episodes_df, summary_df_updated, on='episode', how='left').fillna(0)
    full_summary_df = pd.merge(profits_per_episode, full_summary_df, on='episode', how='left').fillna(0)
    full_summary_df['Time within boundary (%)'] = 100- full_summary_df['Time within boundary (%)']

    return full_summary_df

def calculate_twb(dataframes, labels):
    """
    Calculate the time boundary are violated for each state boundary.
    Given a list of dataframes, with dataframe for each controller.

    Args:
        dataframes (List[pd.DataFrame]): evaluation data
        labels (List[str]): labels for each controller

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: the time within boundary and confidence intervals
    """
    twb_df = pd.DataFrame()
    twb_df_ci = pd.DataFrame()
    N = dataframes[0]['episode'].unique().shape[0]
    for j, df in enumerate(dataframes):
        vars = ['CO2 violation', 'Temperature violation', 'Humidity violation']
        violations = [aggregate_data(df, var) for var in vars]

        twb = np.array([violations[i]['Time within boundary (%)'].mean() for i in range(len(vars))])
        df_twb = pd.DataFrame({labels[j]: twb,}, index=vars)
        twb_df = pd.concat([twb_df, df_twb], axis=1)
        
        cis = [ci(violations[i]['Time within boundary (%)'].std(), N) for i in range(len(vars))]
        df_twb_ci = pd.DataFrame({labels[j]: cis,}, index=vars)
        twb_df_ci = pd.concat([twb_df_ci, df_twb_ci], axis=1)
    return twb_df.T, twb_df_ci.T



def ci(std, n, z=2.576):
    """Computes the confidence interval for a given standard deviation and sample size.
    Args:
        std ([float, np.ndarray]): the standard deviation
        n (int): the sample size
        z (float): the z-score for the confidence interval
    
    Returns:
        ([float, np.ndarray]): the confidence interval
    """
    return z*std/np.sqrt(n)

In [ ]:
def generate_barplots(dataframes, xlabels, colors):
    """
    Generates bar plots for the profits of each controller.
    Given list of dataframes, xlabels and colors.
    Each instance corresponds to a controller.

    Args:
        dataframes (List[pd.DataFrame]): evaluation data
        xlabels (List[str]): xlabels to plot
        colors (List[str]): colors for each controller
    """
    fig, ax = plt.subplots(dpi=120)
    n = dataframes[0].shape[0]  # Assuming all dataframes have the same shape

    heights = []
    errors = []
    for i, df in enumerate(dataframes):
        heights.append(df['Profits'].mean())
        errors.append(ci(df['Profits'].std(), n))

    ax.set_ylabel(r'$r_{0:N}^{\mathrm{profit}}$')
    ax.bar(xlabels, heights, width=0.6, color=colors, edgecolor='black')
    ax.errorbar(xlabels, heights, yerr=errors, capsize=5, linestyle='None', color='black', fmt='')

    fig.tight_layout()
    fig.savefig('profits.svg') 
    plt.show()



In [ ]:
def generate_grouped_bar_plot(twb_df, twb_df_ci, xlabels, colors):
    """
    Making a grouped bar plot for the time within boundary (TWB) for state constraints.
    For each controller, the three TWB for CO2, Temperature and Humidity are plotted.

    Args:
        twb_df (pd.DataFrame): Mean time within boundary for each boundary
        twb_df_ci (pd.DataFrame): Confidence intervals for the time within boundary
        xlabels (List[str]): xlabels for the plot
        colors (List[str]): colors for each controller
    """
    co2color = cm.tokyoS(4)
    tcolor = cm.tokyoS(9)
    humcolor = cm.tokyoS(6)

    fig, ax = plt.subplots(dpi=120)

    index = np.arange(len(xlabels))

    bar_width = 0.2

    bars1 = ax.bar(index - bar_width, twb_df['CO2 violation'], bar_width, edgecolor='black', color=co2color, label=r'CO$_2$')
    ax.errorbar(index - bar_width, twb_df['CO2 violation'], yerr=twb_df_ci["CO2 violation"], capsize=5, linestyle='None', color='black', fmt='')

    bars2 = ax.bar(index, twb_df['Temperature violation'], bar_width, edgecolor='black', color=tcolor, label='Temperature')
    ax.errorbar(index, twb_df['Temperature violation'], yerr=twb_df_ci["Temperature violation"], capsize=5, linestyle='None', color='black', fmt='')

    bars3 = ax.bar(index + bar_width, twb_df['Humidity violation'], bar_width, edgecolor='black', color=humcolor, label='Humidity')
    ax.errorbar(index + bar_width, twb_df['Humidity violation'], yerr=twb_df_ci["Humidity violation"], capsize=5, linestyle='None', color='black', fmt='')

    ax.set_xticks(index)
    ax.set_xticklabels(xlabels)
    ax.set_ylabel('Time within boundary (%)')
    ax.legend()
    # ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    fig.tight_layout()
    fig.savefig('twb.svg')
    plt.show()


In [ ]:
def plot_avg_day(df, variable, fig, ax, label, linestyle, color, cumsum=False):
    """
    Plot the average value of a variable over the day.
    Taken the average over all days in the evaluation data set, df.

    Args:
        df (pd.DataFrame): the evaluation data
        variable (str): variable to plot
        fig (plt.Figure): figure to plot on
        ax (plt.ax): axis to plot on
        label (str): label for the plot
        linestyle (str): linestyle for the plot
        color (str): color for the variable
        cumsum (bool, optional): whether to plot the cumulative sum over the day. Defaults to False.

    Returns:
        plt.Figure, plt.ax: the figure and axis
    """
    Ndays = 600
    time_step_means = df.groupby('TimeOfDay')[variable].mean().reset_index()
    # time_step_stds = df.groupby('TimeOfDay')[variable].var().reset_index()
    time_step_ci = ci(df.groupby('TimeOfDay')[variable].std().values, Ndays)
    # Convert 'TimeOfDay' to minutes past midnight for plotting
    time_step_means['MinutesPastMidnight'] = time_step_means['TimeOfDay'].apply(lambda t: t.hour + t.minute/60)
    time_step_means['MinutesPastMidnight'] 
    
    # Plotting with the modified time format
    
    if cumsum:
        time_step_means[variable] = time_step_means[variable].cumsum()
        print(time_step_means[variable].iloc[-1])
        time_step_ci
    ax.plot(time_step_means['MinutesPastMidnight'], time_step_means[variable], linestyle=linestyle, linewidth=4, color=color, label=label)
    ax.fill_between(time_step_means['MinutesPastMidnight'], time_step_means[variable]-time_step_ci, time_step_means[variable]+time_step_ci, color=color, alpha=0.3)


    ax.set_xlabel('Hour of the day')
    ax.set_ylabel(variable)
    ax.set_xticks(np.arange(0, 25, 6))
    return fig, ax

In [ ]:
def adjust_time(df):
    """
    Add time of day to the evaluation data as column 'TimeOfDay'.

    Args:
        df (pd.DataFrame): the evaluation data

    Returns:
        pd.DataFrame: evaluation data with 'TimeOfDay' column
    """
    # compute max fruit harvest
    max_fruit_gr = 0.328*1e-6 # kg [DW] m-2 s-1
    delta_t = 300
    max_fruit_gr *= delta_t
    max_fruit_gr

    df['Time'] = df['Time'].map(lambda x: str(x)[:-6])
    df["Fruit harvest norm"] = df["Fruit harvest"] / max_fruit_gr
    df["Fruit harvest FW"] = df["Fruit harvest"] / 0.0627
    # df["Cumulative Fruit harvest norm"] = df["Fruit harvest norm"].cumsum()
    # df["Cumulative Fruit Harvest"] = df["Fruit harvest"].cumsum()
    df['Date'] = pd.to_datetime(df['Time'])
    df['TimeOfDay'] = df['Date'].dt.time
    return df


## Loading and processing the data 

1. Load the data for three controllers: 
- 2 RL-controllers:
    - multiplicative reward
    - additive reward
- Rule-based controller.

2. Compute the statisctics for the data

In [ ]:
# load data as dataframes, adjust time and save them in a list
multiplicative_path =  '../data/benchmark/test/multiplicative-0.99'
additive_path =  '../data/benchmark/test/additive-0.99'

multiplicative_pen_df = load_data(multiplicative_path, 'sweet-energy-9-60-best.csv')
additive_pen_df = load_data(additive_path, 'helpful-sky-20-60-best.csv')
rb_df = load_data('../data/benchmark/test/rule-based', 'benchmark-60-last.csv')

dfs = [additive_pen_df, multiplicative_pen_df, rb_df]
dfs = [adjust_time(df) for df in dfs]

xlabels = ['Additive', 'Multiplicative', 'Rule-based']

# compute stats
profits_per_episode = [compute_profit_eps(df) for df in dfs]
twb_df, twb_df_ci = calculate_twb(dfs, xlabels)

## Do comparability tests

In [ ]:
normaltest(profits_per_episode[0]['Profits']), normaltest(profits_per_episode[1]['Profits'])

In [ ]:
mannwhitneyu(profits_per_episode[0]['Profits'], profits_per_episode[1]['Profits'], method='exact')

In [ ]:
mannwhitneyu(profits_per_episode[1]['Profits'], profits_per_episode[2]['Profits'], method='exact')

In [ ]:
mannwhitneyu(profits_per_episode[0]['Profits'], profits_per_episode[2]['Profits'], method='exact')


In [ ]:
mannwhitneyu(profits_per_episode[1]['Profits'], profits_per_episode[2]['Profits'], method='exact')

## Generate Bar plots

In [ ]:
# plot profits and TWB with confidence intervals
colors = [cm.tokyoS(4), cm.tokyoS(6), cm.tokyoS(9)]
generate_barplots(profits_per_episode, xlabels,colors)
generate_grouped_bar_plot(twb_df, twb_df_ci, xlabels, colors)

## Daily averages of state and control variables

1. Plots the state variables
2. Plots the control variables

In [ ]:
# State variables to plot

state_variables = ['Air Temperature', 'CO2 concentration', 'Humidity', 'Fruit harvest FW']
labels = ['Additive', 'Multiplicative', 'Rule-based']
ylabels = [r'Temperature ($^\circ$C)', '$\mathrm{CO_2}$ (ppm)', '$\mathrm{RH}}$ (%)', 'Fruit harvest (kg/m$^2$)']
ylim = [(16, 26), (500, 800), (60, 100), (0, 0.3)]
for i, variable in enumerate(state_variables):
    fig, ax = plt.subplots(dpi=120)
    if 'Fruit' in variable:
        plot_avg_day(dfs[0], variable, fig, ax, labels[0], '-', colors[0], cumsum=True)
        plot_avg_day(dfs[1], variable, fig, ax, labels[1], '-', colors[1], cumsum=True)
        plot_avg_day(dfs[2], variable, fig, ax, labels[2], '-', colors[2], cumsum=True)
    else:
        plot_avg_day(dfs[0], variable, fig, ax, labels[0], '-', colors[0])
        plot_avg_day(dfs[1], variable, fig, ax, labels[1], '-', colors[1])
        plot_avg_day(dfs[2], variable, fig, ax, labels[2], '-', colors[2])
    ax.set_ylabel(ylabels[i])
    ax.set_ylim(ylim[i])
    fig.savefig(f'../figures/24h-mean-{variable}-2.svg', format='svg')
ax.legend()

fig.savefig(f'../figures/24h-mean-{variable}-2.svg', format='svg')


In [ ]:
rlhar = 0.25661656214864836
rbhar = 0.26912746541713334
rbhar-rlhar

In [ ]:
# plotting control variables
control_variables= ['uCO2', 'uBoil', 'uVent', 'uThScr']
labels = ['Additive', 'Multiplicative', 'Rule-based']
ylabels = ['$u_{\mathrm{CO_2}}$', '$u_{\mathrm{boil}}$ (ppm)', '$u_{\mathrm{vent}}$ (%)', '$u_{\mathrm{thScr}}$ (%)']
for i, variable in enumerate(control_variables):
    fig, ax = plt.subplots(dpi=120)
    plot_avg_day(dfs[0], variable, fig, ax, labels[0], '-', colors[0])
    plot_avg_day(dfs[1], variable, fig, ax, labels[1], '-', colors[1])
    plot_avg_day(dfs[2], variable, fig, ax, labels[2], '-', colors[2])
    ax.set_ylabel(ylabels[i])
    ax.set_ylim(0, 1)
    fig.savefig(f'../figures/24h-mean-{variable}-2.svg', format='svg')
ax.legend()
fig.savefig(f'../figures/24h-mean-{variable}-2.svg', format='svg')


## From here on unused figures...

- Seasonal plots for the performance over months.
- 

In [ ]:
def calc_svp(temp):
    return 610.78*np.exp(17.2694*temp/(temp+238.3))


def calc_vpd(svp, rh):
    return svp * (1 - rh/100)

def sort_runs_by_profit(profits_per_episode):
    return [df.sort_values(by='Profits', ascending=False) for df in profits_per_episode]

In [ ]:
def compute_seasonal_profits(df):
    df['Date'] = pd.to_datetime(df['Time'])
    df['Month'] = df['Date'].dt.month
    # convert numeric months to string
    df['Cumulative profit'] = df.groupby('episode')['Profits'].cumsum()

    # extract final profit for each episode
    final_profits = df.groupby('episode')[['Cumulative profit', 'Month', 'Date']].last().reset_index()
    
    seasonal_profits = final_profits.groupby(['Month'])['Cumulative profit'].mean()
    # print(seasonal_profits)
    seasonal_std = final_profits.groupby(['Month'])['Cumulative profit'].std()
    seasonality = pd.DataFrame({'Mean': seasonal_profits, 'Std': seasonal_std})

    # seasonality.reset_index(inplace=True, drop=False)
    # convert month column as string type

    seasonality.reset_index(inplace=True, drop=False)
    seasonality['Month'] = pd.to_datetime(seasonality['Month'], format='%m').dt.month_name().str.slice(stop=3)
    return seasonality


In [ ]:
seasonality = compute_seasonal_profits(dfs[0])
seasonality_rb = compute_seasonal_profits(dfs[1])


In [ ]:


fig, ax = plt.subplots(dpi=120)
bar_width = 0.4
ax.bar(seasonality.index-bar_width/2, seasonality['Mean'], yerr=seasonality['Std'], width=bar_width, capsize=5, color=colors[0], label='RL', edgecolor='black')
ax.bar(seasonality_rb.index+bar_width/2, seasonality_rb['Mean'], yerr=seasonality_rb['Std'], width=bar_width, capsize=5, color=colors[1], label='Rule-based', edgecolor='black')

ax.set_ylabel(r'$r_{T}^{\mathrm{profit}}$')
# set xticks to month
ax.set_xticks(seasonality.index)
ax.set_xticklabels(seasonality['Month'], ha='center')

# create a list of months
fig.savefig('../figures/seasonal-profits.svg', format='svg')


In [ ]:
# sorted_multi  = profits_per_episode[0].sort_values(by='Profits', ascending=False)
# sorted_rb  = profits_per_episode[1].sort_values(by='Profits', ascending=False)

# sorted_multi#

In [ ]:
# sorted_rb

# Plots that show a single episode for controllers. Unpolished plots. 

In [ ]:
episode = 0

rl_df = dfs[0]
rb_df = dfs[1]

fig = plt.figure(figsize=(10, 20))
ax1 = fig.add_subplot(811)
ax2 = fig.add_subplot(812)
ax3 = fig.add_subplot(813)
ax4 = fig.add_subplot(814)
ax5 = fig.add_subplot(815)
ax6 = fig.add_subplot(816)
ax7 = fig.add_subplot(817)
ax8 = fig.add_subplot(818)

rl_df[rl_df['episode'] == episode].plot(x='Time', y='Air Temperature', kind='line', color=colors[0], ax=ax1, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='Air Temperature', kind='line', color=colors[1], ax=ax1, label='Rule-based')
ax1.set_ylabel('Air Temperature')

rl_df[rl_df['episode'] == episode].plot(x='Time', y='uVent', kind='line', color=colors[0], ax=ax2, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='uVent', kind='line', color=colors[1], ax=ax2, label='Rule-based')
ax2.set_ylabel('Vent')

rl_df[rl_df['episode'] == episode].plot(x='Time', y='uBoil', kind='line', color=colors[0], ax=ax3, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='uBoil', kind='line', color=colors[1], ax=ax3, label='Rule-based')
ax3.set_ylabel('Boil')

rl_df[rl_df['episode'] == episode].plot(x='Time', y='uThScr', kind='line', color=colors[0], ax=ax4, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='uThScr', kind='line', color=colors[1], ax=ax4, label='Rule-based')
ax4.set_ylabel('Thermal Screen')

rl_df[rl_df['episode'] == episode].plot(x='Time', y='uLamp', kind='line', color=colors[0], ax=ax5, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='uLamp', kind='line', color=colors[1], ax=ax5, label='Rule-based')
ax5.set_ylabel('Lamp')


rl_df[rl_df['episode'] == episode].plot(x='Time', y='uCO2', kind='line', color=colors[0], ax=ax6, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='uCO2', kind='line', color=colors[1], ax=ax6, label='Rule-based')
ax6.set_ylabel('CO2')

rl_df[rl_df['episode'] == episode].plot(x='Time', y='PAR', kind='line', color=colors[0], ax=ax7, label='RL')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='PAR', kind='line', color=colors[1], ax=ax7, label='Rule-based')
ax7.set_ylabel('PAR')


rl_df[rl_df['episode'] == episode].plot(x='Time', y='Humidity', kind='line', color=colors[0], ax=ax8, label='Rule-based')
rb_df[rb_df['episode'] == episode].plot(x='Time', y='Humidity', kind='line', color=colors[1], ax=ax8, label='RL')
ax8.set_ylabel('RH')


# rl_df[rl_df['episode'] == episode].plot(x='Time', y='uCO2', kind='line', color=colors[0], ax=ax6, label='Rule-based')
# rb_df[rb_df['episode'] == episode].plot(x='Time', y='uCO2', kind='line', color=colors[0], ax=ax6, label='RL')

# remove x ticks from axis
ax1.set_xticks([])
ax2.set_xticks([])
ax3.set_xticks([])
ax4.set_xticks([])
ax5.set_xticks([])
ax6.set_xticks([])
ax7.set_xticks([])
# skip legend
ax1.legend().remove()
ax2.legend().remove()
ax3.legend().remove()
ax4.legend().remove()
ax5.legend().remove()
ax6.legend().remove()
ax7.legend().remove()
# ax8.legend().remove()

# rotate x labels
ax8.set_xticklabels(ax8.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()
# rl_df[rl_df['episode'] == episode].plot(x='Time', y='Air Temperature', kind='scatter', colormap='viridis', c=rl_df[rl_df['episode'] == episode]['Fruit harvest norm'], ax=ax)
# rb_df[rb_df['episode'] == episode].plot(x='Time', y='Fruit weight', kind='scatter', colormap='viridis', c=rb_df[rb_df['episode'] == episode]['Fruit harvest norm'], ax=ax)


In [ ]:
fig, axes = plt.subplots(1,4 , figsize=(20, 5), dpi=120)
rb_df['Cumulative Profit'] = rb_df.groupby('episode')['Profits'].cumsum()
rl_df['Cumulative Profit'] = rl_df.groupby('episode')['Profits'].cumsum()

rb_df['Cumulative Harvest'] = rb_df.groupby('episode')['Fruit harvest'].cumsum()
rl_df['Cumulative Harvest'] = rl_df.groupby('episode')['Fruit harvest'].cumsum()

rb_df['Cumulative co2'] = rb_df.groupby('episode')['CO2 resource'].cumsum()
rl_df['Cumulative co2'] = rl_df.groupby('episode')['CO2 resource'].cumsum()

rb_df['Cumulative gas'] = rb_df.groupby('episode')['Gas resource'].cumsum()
rl_df['Cumulative gas'] = rl_df.groupby('episode')['Gas resource'].cumsum()


# episode=9

cumulatives = ['Cumulative Profit', 'Cumulative Harvest', 'Cumulative co2', 'Cumulative gas']

for i, cost in enumerate(cumulatives):
    rl_df[rl_df['episode'] == episode].plot(x='Time', y=cumulatives[i], kind='line', color=colors[0],ax=axes[i], label='RL')
    rb_df[rb_df['episode'] == episode].plot(x='Time', y=cumulatives[i], kind='line', color=colors[1], ax=axes[i], label='Rule-based')
    axes[i].set_ylabel(cost)
    axes[i].legend().remove()
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45)

